# FinBERT Hyperparameter Sweep 

This notebook extends the standard FinBERT training with:
- Hyperparameter optimization using W&B sweeps
- Automated experiment tracking
- Comparison of different configurations

## Prerequisites
```bash
pip install wandb
wandb login
```


## 1. Setup and Imports

In [1]:
from pathlib import Path
import shutil
import os
import logging
import sys
import numpy as np
sys.path.append('..')

from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification
import torch
from torch.nn import CrossEntropyLoss

from finbert.finbert import *
import finbert.utils as tools

# Weights & Biases
import wandb

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

print("Imports loaded successfully")
print(f"Project directory: {project_dir}")

Imports loaded successfully
Project directory: /home/si2449/hpml-project/pipelines/finBERT


In [2]:
from pathlib import Path
import shutil
import os
import logging
import sys
import numpy as np
sys.path.append('..')

from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification
import torch
from torch.nn import CrossEntropyLoss

from finbert.finbert import *
import finbert.utils as tools

# Weights & Biases
import wandb

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

print("Imports loaded successfully")
print(f"Project directory: {project_dir}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Imports loaded successfully
Project directory: /home/si2449/hpml-project/pipelines/finBERT


## 2. Configuration

Set up paths and W&B project name.

In [3]:
# Paths
cl_path = project_dir/'models'/'sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

# W&B Configuration
WANDB_PROJECT = "Project-Runs"
WANDB_ENTITY = None  

print(f"Model path: {cl_path}")
print(f"Data path: {cl_data_path}")
print(f"W&B Project: {WANDB_PROJECT}")

Model path: /home/si2449/hpml-project/pipelines/finBERT/models/sentiment
Data path: /home/si2449/hpml-project/pipelines/finBERT/data/sentiment_data
W&B Project: Project-Runs


## 3. Define Sweep Configuration

This defines the hyperparameter search space. W&B will automatically try different combinations.


In [4]:
sweep_config = {
    'method': 'bayes',  # 'grid', 'random', or 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 5e-5
        },
        'num_train_epochs': {
            'values': [3, 4, 5, 6]
        },
        'train_batch_size': {
            'values': [16, 32, 64]
        },
        'warm_up_proportion': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.3
        },
        'max_seq_length': {
            'values': [48, 64, 96]
        },
        # Advanced parameters
        'discriminate': {
            'values': [True, False]
        },
        'gradual_unfreeze': {
            'values': [True, False]
        }
    }
}

print("Sweep configuration created")
print(f"  Method: {sweep_config['method']}")
print(f"  Optimization metric: {sweep_config['metric']['name']}")


Sweep configuration created
  Method: bayes
  Optimization metric: val_loss


## 4. Training Function with W&B Integration

This wraps existing training code with W&B logging.


In [5]:
def train_with_config(config=None):
    """
    Training function that W&B will call with different hyperparameters.
    """
    # Initialize W&B run
    with wandb.init(config=config):
        # Get hyperparameters from W&B
        config = wandb.config
        
        print(f"\n{'='*80}")
        print(f"Starting training run with config:")
        print(f"  Learning rate: {config.learning_rate}")
        print(f"  Epochs: {config.num_train_epochs}")
        print(f"  Batch size: {config.train_batch_size}")
        print(f"  Warmup: {config.warm_up_proportion}")
        print(f"  Max seq length: {config.max_seq_length}")
        print(f"{'='*80}\n")
        
        # Clean previous model directory
        model_path = project_dir / 'models' / 'sentiment' / f'sweep_{wandb.run.id}'
        try:
            shutil.rmtree(model_path)
        except:
            pass
        
        # Create BERT model
        bertmodel = AutoModelForSequenceClassification.from_pretrained(
            'bert-base-uncased', cache_dir=None, num_labels=3
        )
        
        # Create FinBERT config with hyperparameters from sweep
        finbert_config = Config(
            data_dir=cl_data_path,
            bert_model=bertmodel,
            num_train_epochs=config.num_train_epochs,
            model_dir=model_path,
            max_seq_length=config.max_seq_length,
            train_batch_size=config.train_batch_size,
            learning_rate=config.learning_rate,
            output_mode='classification',
            warm_up_proportion=config.warm_up_proportion,
            local_rank=-1,
            discriminate=config.discriminate,
            gradual_unfreeze=config.gradual_unfreeze
        )
        
        # Initialize FinBERT
        finbert = FinBert(finbert_config)
        finbert.base_model = 'bert-base-uncased'
        finbert.config.discriminate = config.discriminate
        finbert.config.gradual_unfreeze = config.gradual_unfreeze
        finbert.prepare_model(label_list=['positive', 'negative', 'neutral'])
        
        # Load data
        train_data = finbert.get_data('train')
        test_data = finbert.get_data('test')
        
        # Create model
        model = finbert.create_the_model()
        
        # Train with W&B logging
        trained_model = train_with_wandb_logging(
            finbert, train_data, model, test_data
        )
        
        # Final evaluation
        results = finbert.evaluate(examples=test_data, model=trained_model)
        results['prediction'] = results.predictions.apply(lambda x: np.argmax(x, axis=0))
        
        # Calculate metrics
        metrics = calculate_metrics(results, finbert)
        
        # Log final metrics to W&B
        wandb.log({
            'final_test_loss': metrics['loss'],
            'final_test_accuracy': metrics['accuracy'],
            'final_f1_positive': metrics['f1_positive'],
            'final_f1_negative': metrics['f1_negative'],
            'final_f1_neutral': metrics['f1_neutral'],
            'final_f1_macro': metrics['f1_macro']
        })
        
        print(f"\n{'='*80}")
        print(f"Final Results:")
        print(f"  Test Loss: {metrics['loss']:.4f}")
        print(f"  Test Accuracy: {metrics['accuracy']:.4f}")
        print(f"  Macro F1: {metrics['f1_macro']:.4f}")
        print(f"{'='*80}\n")
        
        return metrics


def train_with_wandb_logging(finbert, train_data, model, test_data):
    """
    Modified training loop with W&B logging.
    """
    validation_examples = finbert.get_data('validation')
    global_step = 0
    finbert.validation_losses = []
    
    train_dataloader = finbert.get_loader(train_data, 'train') 
    model.train()
    step_number = len(train_dataloader)
    
    i = 0
    best_val_loss = float('inf')
    
    for epoch in trange(int(finbert.config.num_train_epochs), desc="Epoch"):
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        
        for step, batch in enumerate(tqdm(train_dataloader, desc='Iteration')):
            
            # Gradual unfreezing logic
            if (finbert.config.gradual_unfreeze and i == 0):
                for param in model.bert.parameters():
                    param.requires_grad = False
            
            if (step % (step_number // 3)) == 0:
                i += 1
            
            if (finbert.config.gradual_unfreeze and i > 1 and i < finbert.config.encoder_no):
                for k in range(i - 1):
                    try:
                        for param in model.bert.encoder.layer[finbert.config.encoder_no - 1 - k].parameters():
                            param.requires_grad = True
                    except:
                        pass
            
            if (finbert.config.gradual_unfreeze and i > finbert.config.encoder_no + 1):
                for param in model.bert.embeddings.parameters():
                    param.requires_grad = True
            
            batch = tuple(t.to(finbert.device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, agree_ids = batch
            
            logits = model(input_ids, attention_mask, token_type_ids)[0]
            weights = finbert.class_weights.to(finbert.device)
            
            if finbert.config.output_mode == "classification":
                loss_fct = CrossEntropyLoss(weight=weights)
                loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))
            elif finbert.config.output_mode == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), label_ids.view(-1))
            
            if finbert.config.gradient_accumulation_steps > 1:
                loss = loss / finbert.config.gradient_accumulation_steps
            else:
                loss.backward()
            
            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            
            if (step + 1) % finbert.config.gradient_accumulation_steps == 0:
                if finbert.config.fp16:
                    lr_this_step = finbert.config.learning_rate * warmup_linear(
                        global_step / finbert.num_train_optimization_steps, 
                        finbert.config.warm_up_proportion
                    )
                    for param_group in finbert.optimizer.param_groups:
                        param_group['lr'] = lr_this_step
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                finbert.optimizer.step()
                finbert.scheduler.step()
                finbert.optimizer.zero_grad()
                global_step += 1
                
                # Log to W&B every N steps
                if global_step % 10 == 0:
                    wandb.log({
                        'train_loss': tr_loss / nb_tr_steps,
                        'learning_rate': finbert.optimizer.param_groups[0]['lr'],
                        'epoch': epoch,
                        'step': global_step
                    })
        
        # Validation at end of epoch
        validation_loader = finbert.get_loader(validation_examples, 'eval')
        model.eval()
        
        valid_loss, valid_accuracy = 0, 0
        nb_valid_steps, nb_valid_examples = 0, 0
        
        for input_ids, attention_mask, token_type_ids, label_ids, agree_ids in tqdm(validation_loader, desc="Validating"):
            input_ids = input_ids.to(finbert.device)
            attention_mask = attention_mask.to(finbert.device)
            token_type_ids = token_type_ids.to(finbert.device)
            label_ids = label_ids.to(finbert.device)
            agree_ids = agree_ids.to(finbert.device)
            
            with torch.no_grad():
                logits = model(input_ids, attention_mask, token_type_ids)[0]
                
                if finbert.config.output_mode == "classification":
                    loss_fct = CrossEntropyLoss(weight=weights)
                    tmp_valid_loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))
                elif finbert.config.output_mode == "regression":
                    loss_fct = MSELoss()
                    tmp_valid_loss = loss_fct(logits.view(-1), label_ids.view(-1))
                
                valid_loss += tmp_valid_loss.mean().item()
                nb_valid_steps += 1
        
        valid_loss = valid_loss / nb_valid_steps
        finbert.validation_losses.append(valid_loss)
        
        # Log validation metrics to W&B
        wandb.log({
            'val_loss': valid_loss,
            'epoch': epoch,
            'best_val_loss': min(finbert.validation_losses)
        })
        
        print(f"Epoch {epoch}: Validation loss = {valid_loss:.4f}")
        
        # Save best model
        if valid_loss == min(finbert.validation_losses):
            try:
                os.remove(finbert.config.model_dir / ('temporary' + str(best_model)))
            except:
                pass
            torch.save({'epoch': str(epoch), 'state_dict': model.state_dict()},
                       finbert.config.model_dir / ('temporary' + str(epoch)))
            best_model = epoch
            best_val_loss = valid_loss
    
    # Load best model
    checkpoint = torch.load(finbert.config.model_dir / ('temporary' + str(best_model)))
    model.load_state_dict(checkpoint['state_dict'])
    
    # Save final model
    model_to_save = model.module if hasattr(model, 'module') else model
    output_model_file = os.path.join(finbert.config.model_dir, 'pytorch_model.bin')
    torch.save(model_to_save.state_dict(), output_model_file)
    
    # Clean up temporary files
    try:
        os.remove(finbert.config.model_dir / ('temporary' + str(best_model)))
    except:
        pass
    
    return model


def calculate_metrics(results, finbert):
    """
    Calculate comprehensive metrics for evaluation.
    """
    from sklearn.metrics import f1_score, precision_score, recall_score
    
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(
        torch.tensor(list(results['predictions'])),
        torch.tensor(list(results['labels']))
    )
    
    accuracy = (results['labels'] == results['prediction']).sum() / results.shape[0]
    
    # Calculate per-class F1 scores
    f1_scores = f1_score(results['labels'], results['prediction'], average=None)
    f1_macro = f1_score(results['labels'], results['prediction'], average='macro')
    
    return {
        'loss': loss.item(),
        'accuracy': accuracy,
        'f1_positive': f1_scores[0],
        'f1_negative': f1_scores[1],
        'f1_neutral': f1_scores[2],
        'f1_macro': f1_macro
    }


print("Training functions defined")

Training functions defined


## 5. Initialize and Run Sweep

This will start the hyperparameter sweep. W&B will automatically try different combinations.


In [6]:
# Initialize the sweep
sweep_id = wandb.sweep(
    sweep_config, 
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY
)

print(f"Sweep initialized with ID: {sweep_id}")
print(f" View at: https://wandb.ai/{WANDB_ENTITY or 'your-username'}/{WANDB_PROJECT}/sweeps/{sweep_id}")

Create sweep with ID: jwcyzd18
Sweep URL: https://wandb.ai/si2449-columbia-university/Project-Runs/sweeps/jwcyzd18
Sweep initialized with ID: jwcyzd18
 View at: https://wandb.ai/your-username/Project-Runs/sweeps/jwcyzd18


In [7]:
# Run the sweep
# count: number of runs to execute
wandb.agent(sweep_id, function=train_with_config, count=10)

print("\n" + "="*80)
print("SWEEP COMPLETED")
print("="*80)
print(f"View results at: https://wandb.ai/{WANDB_ENTITY or 'your-username'}/{WANDB_PROJECT}/sweeps/{sweep_id}")


wandb: Agent Starting Run: adoux3xi with config:
wandb: 	discriminate: False
wandb: 	gradual_unfreeze: True
wandb: 	learning_rate: 4.6061389106252933e-05
wandb: 	max_seq_length: 96
wandb: 	num_train_epochs: 5
wandb: 	train_batch_size: 16
wandb: 	warm_up_proportion: 0.14560584591904005
wandb: Currently logged in as: si2449 (si2449-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Starting training run with config:
  Learning rate: 4.6061389106252933e-05
  Epochs: 5
  Batch size: 16
  Warmup: 0.14560584591904005
  Max seq length: 96



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/19/2025 22:54:36 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


...

Epoch 3: Validation loss = 0.9345


Epoch: 100%|██████████| 4/4 [02:45<00:00, 41.38s/it]
12/19/2025 23:22:46 - INFO - finbert.utils -   *** Example ***
12/19/2025 23:22:46 - INFO - finbert.utils -   guid: test-1
12/19/2025 23:22:46 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/19/2025 23:22:46 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/19/2025 23:22:46 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


Final Results:
  Test Loss: 0.9610
  Test Accuracy: 0.5835
  Macro F1: 0.4776



best_val_loss,█▁▁▁
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆██████████
final_f1_macro,▁
final_f1_negative,▁
final_f1_neutral,▁
final_f1_positive,▁
final_test_accuracy,▁
final_test_loss,▁
learning_rate,▂▆██▇▆▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
+2,...



SWEEP COMPLETED
View results at: https://wandb.ai/your-username/Project-Runs/sweeps/jwcyzd18


## 6. Analyze Results


In [ ]:
# Best config from W&B sweep (absurd-sweep-6)  [tag: hyper-parameter-sweep-baseline]
best_config = {
    'learning_rate': 0.0000350823040129893,
    'num_train_epochs': 5,
    'train_batch_size': 32,
    'warm_up_proportion': 0.13649624266391375,
    'max_seq_length': 64,
    'discriminate': False,
    'gradual_unfreeze': False
}

# train_with_config(config=best_config)


## Best hyperparameters (from W&B sweep)

We select **`absurd-sweep-6`** as the best run from the **baseline sweep** (tag: `hyper-parameter-sweep-baseline`) because it achieves the strongest overall generalization and test performance:

- **Lowest validation loss**: `best_val_loss = 0.4374` (best generalization signal during tuning)
- **Best test performance overall**:
  - `final_test_accuracy = 0.8031` (highest)
  - `final_test_loss = 0.4682` (lowest)
  - `final_f1_macro = 0.7807` (highest)

**Best configuration (optimal hyperparameters):**
- `learning_rate = 3.50823040129893e-05`
- `num_train_epochs = 5`
- `train_batch_size = 32`
- `warm_up_proportion = 0.13649624266391375`
- `max_seq_length = 64`
- `discriminate = False`
- `gradual_unfreeze = False`

**Why this configuration likely works well:** the learning rate (~`3.5e-5`) is high enough to adapt quickly, while the warmup (~`0.136`) stabilizes early updates. Keeping `max_seq_length = 64` preserves sufficient context, and disabling `discriminate`/`gradual_unfreeze` suggests a simpler fine-tuning schedule generalized better than more complex layer-wise strategies on this dataset.
